In [ ]:
!nvidia-smi

In [ ]:
!pip install librosa evaluate datasets jiwer gcsfs accelerate transformers==4.37.2

In [ ]:
import librosa, torch, evaluate, os
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import Seq2SeqTrainingArguments

In [ ]:
print('Initializing...')
metric = evaluate.load("wer")

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large", language="Bengali", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-large", language="Bengali", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
def prepare_dataset(batch):
    audio_array, sampling_rate = librosa.load(batch["path"], sr=16000, mono=True)
    batch["input_features"] = feature_extractor(audio_array, sampling_rate=sampling_rate).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
ds = load_dataset('csv', data_files={'train': ['train.csv'], 'test': 'test.csv'})

In [ ]:
print(ds)
ds = ds.map(prepare_dataset, num_proc=None)
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="ckpt/whisper-large-bn-snt",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=38,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=255,
    # save_strategy="epoch",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("ckpt/whisper-large-bn-snt/checkpoint-2000")

In [ ]:
evaluator = Seq2SeqTrainer(
    args=training_args,
    model=model,
    # train_dataset=ds["train"],
    eval_dataset=ds["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
evaluator.evaluate(eval_dataset=ds["test"])

In [ ]:
evaluator.predict(test_dataset=ds["test"]).predictions

In [ ]:
transcription = processor.batch_decode(evaluator.predict(test_dataset=ds["test"], num_beams=5).predictions, skip_special_tokens=True)

In [ ]:
for sent in transcription:
    print(sent)